In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import manhattan_distances
from sklearn.metrics.pairwise import euclidean_distances

In [4]:
data_set = pd.read_csv("C:/Users/AbdulRahman/Desktop/grad_folders/MAIN_GRAD/Travia/Implementation/AI/DataSet/lateset Dataset Main/pre_processed_df_without.csv")    

In [5]:
data_set.columns

Index(['latitude', 'longitude', 'stars', 'review_count',
       'attributes_RestaurantsPriceRange2', 'attributes_GoodForKids',
       'attributes_Ambience_touristy', 'attributes_Ambience_romantic',
       'attributes_Ambience_intimate', 'attributes_Ambience_trendy',
       'attributes_Ambience_classy', 'attributes_Ambience_casual',
       'Bars_Night', 'Beauty_Health_Care', 'Cafes', 'GYM',
       'Restaurants_Cuisines', 'Shops'],
      dtype='object')

In [7]:
data_set.head()

,latitude,longitude,stars,review_count,attributes_RestaurantsPriceRange2,attributes_GoodForKids,attributes_Ambience_touristy,attributes_Ambience_romantic,attributes_Ambience_intimate,attributes_Ambience_trendy,...,hours_Sunday,attributes_RestaurantsGoodForGroups,attributes_RestaurantsAttire,attributes_NoiseLevel,attributes_GoodForMeal_dessert,attributes_GoodForMeal_latenight,attributes_GoodForMeal_lunch,attributes_GoodForMeal_dinner,attributes_GoodForMeal_brunch,attributes_GoodForMeal_breakfast
0,39.955505,-75.155564,4.0,80,1,1,0,0,0,0,...,7:0-21:0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,39.953949,-75.143226,4.0,245,2,0,0,0,0,0,...,13:30-22:0,True,'casual',u'average',True,NaN,NaN,True,NaN,NaN
2,39.943223,-75.162568,4.5,205,1,1,0,0,0,0,...,NaN,False,u'casual',u'quiet',False,False,True,True,False,False
3,40.006707,-75.289671,3.5,109,1,1,0,0,0,0,...,11:0-0:0,True,u'casual',u'average',False,True,False,True,False,False
4,39.962582,-75.135657,3.5,65,2,1,0,0,0,0,...,11:0-20:0,NaN,NaN,NaN,False,False,False,False,False,False


# feature selection

In [8]:
ds = data_set[
    [
        "stars",

        "attributes_RestaurantsPriceRange2",

        # good for kids or romantic
        "attributes_GoodForKids", # option1

        "attributes_Ambience_romantic", # option2
        "attributes_Ambience_intimate", # option2

        "attributes_Ambience_touristy",

        # trendy or classy or casual 
        "attributes_Ambience_trendy",
        "attributes_Ambience_classy",
        "attributes_Ambience_casual",

        # choose one or many from those 
        "Bars_Night",
        "Beauty_Health_Care",
        "Cafes",
        "GYM",
        "Restaurants_Cuisines",
        "Shops"
    ]
]

In [9]:
ds.head()

,stars,attributes_RestaurantsPriceRange2,attributes_GoodForKids,attributes_Ambience_romantic,attributes_Ambience_intimate,attributes_Ambience_touristy,attributes_Ambience_trendy,attributes_Ambience_classy,attributes_Ambience_casual,Bars_Night,Beauty_Health_Care,Cafes,GYM,Restaurants_Cuisines,Shops
0,4.0,1,1,0,0,0,0,0,1,0,0,1,0,1,0
1,4.0,2,0,0,0,0,0,1,0,0,0,0,0,1,0
2,4.5,1,1,0,0,0,0,0,1,0,0,0,0,1,0
3,3.5,1,1,0,0,0,0,0,1,0,0,0,0,1,0
4,3.5,2,1,0,0,0,0,1,0,0,0,0,0,1,0


In [10]:
def create_user_pref_vector(user_pref):
    user_vector = np.zeros(15, dtype=np.float32)
    user_vector[0] = user_pref["min_stars"]
    user_vector[1] = user_pref["price_range"]
    if user_pref["intimate_or_family"] == "family":
        user_vector[2] = 1
    else:
        user_vector[4] = 1
        user_vector[5] = 1
    user_vector[3] = user_pref["attributes_Ambience_touristy"]
    user_vector[6] = user_pref["attributes_Ambience_trendy"]
    user_vector[7] = user_pref["attributes_Ambience_classy"]
    user_vector[8] = user_pref["attributes_Ambience_casual"]

    for cat in user_pref["category"]:
        user_vector[9 + cat] = 1
    return user_vector

In [11]:
user_pref = {
    #"Bars_Night","Beauty_Health_Care","Cafes","GYM","Restaurants_Cuisines","Shops"
    "category": [0,2,4],
    "min_stars": 3.5,
    "price_range": 3,
    "attributes_Ambience_touristy": 0,
    "attributes_Ambience_trendy": 1,
    "attributes_Ambience_classy": 0,
    "attributes_Ambience_casual": 1,
    "intimate_or_family": 'family'
}

In [12]:
v = create_user_pref_vector(user_pref)

In [13]:
v.reshape(1, -1)

array([[3.5, 3. , 1. , 0. , 0. , 0. , 1. , 0. , 1. , 1. , 0. , 1. , 0. ,
        1. , 0. ]], dtype=float32)

# cosine similairty

In [15]:
def recommend_businesses(user_pref, data, original_df, k=3):
    user_vector = create_user_pref_vector(user_pref)    
    similarities = cosine_similarity(data, user_vector.reshape(1, -1)).flatten()
    top_k_unsorted = np.argpartition(-similarities, k)[:k]
    sorted_indices = top_k_unsorted[np.argsort(similarities[top_k_unsorted])[::-1]]
    recommendations = original_df.iloc[sorted_indices][[
        "name", "stars", "attributes_RestaurantsPriceRange2", "address", "city","attributes_GoodForKids","attributes_Ambience_romantic","attributes_Ambience_intimate","attributes_Ambience_touristy","attributes_Ambience_trendy","attributes_Ambience_classy","attributes_Ambience_casual",
        "Bars_Night", "Beauty_Health_Care", "Cafes", "GYM", "Restaurants_Cuisines", "Shops"
    ]].copy()
    recommendations["similarity"] = similarities[sorted_indices]
    return recommendations

In [17]:
recommend_businesses(user_pref,ds,data_set)

,name,stars,attributes_RestaurantsPriceRange2,address,city,attributes_GoodForKids,attributes_Ambience_romantic,attributes_Ambience_intimate,attributes_Ambience_touristy,attributes_Ambience_trendy,attributes_Ambience_classy,attributes_Ambience_casual,Bars_Night,Beauty_Health_Care,Cafes,GYM,Restaurants_Cuisines,Shops,similarity
13488,Topgolf,3.5,3,500 Cowan St,Nashville,1,0,0,0,1,0,1,1,0,0,0,1,0,0.981480
9960,Stones Throw at Armature Works,4.0,3,1910 North Ola Avenue,Tampa,1,0,0,0,1,0,1,1,0,0,0,1,0,0.979296
5451,Mr and Mrs Crab,4.0,3,2042 Badlands Dr,Brandon,1,0,0,0,1,0,1,1,0,0,0,1,0,0.979296


# manhattan distances

In [18]:
def recommend_businesses_manhattan(user_pref, data, original_df, k=3):
    user_vector = create_user_pref_vector(user_pref)
    
    distances = manhattan_distances(data, user_vector.reshape(1, -1)).flatten()
    
    top_k_unsorted = np.argpartition(distances, k)[:k]
    sorted_indices = top_k_unsorted[np.argsort(distances[top_k_unsorted])]
    
    
    recommendations = original_df.iloc[sorted_indices][[
        "name", "stars", "attributes_RestaurantsPriceRange2", "address", "city",
        "attributes_GoodForKids", "attributes_Ambience_romantic", "attributes_Ambience_intimate",
        "attributes_Ambience_touristy", "attributes_Ambience_trendy", "attributes_Ambience_classy",
        "attributes_Ambience_casual", "Bars_Night", "Beauty_Health_Care", "Cafes",
        "GYM", "Restaurants_Cuisines", "Shops"
    ]].copy()
    
    
    recommendations["manhattan_distance"] = distances[sorted_indices]
    recommendations["similarity_score"] = 1 / (1 + recommendations["manhattan_distance"])    
    return recommendations

In [20]:
recommend_businesses_manhattan(user_pref, ds, data_set)

,name,stars,attributes_RestaurantsPriceRange2,address,city,attributes_GoodForKids,attributes_Ambience_romantic,attributes_Ambience_intimate,attributes_Ambience_touristy,attributes_Ambience_trendy,attributes_Ambience_classy,attributes_Ambience_casual,Bars_Night,Beauty_Health_Care,Cafes,GYM,Restaurants_Cuisines,Shops,manhattan_distance,similarity_score
13488,Topgolf,3.5,3,500 Cowan St,Nashville,1,0,0,0,1,0,1,1,0,0,0,1,0,1.0,0.5
9960,Stones Throw at Armature Works,4.0,3,1910 North Ola Avenue,Tampa,1,0,0,0,1,0,1,1,0,0,0,1,0,1.5,0.4
7703,Wabi Sabi Japanese Fusion Cafe,4.0,2,35583 US Hwy 19 N,Palm Harbor,1,0,0,0,1,0,1,1,0,1,0,1,0,1.5,0.4


# Euclidean Distance

In [21]:
def recommend_businesses_euclidean(user_pref, data, original_df, k=3):
    user_vector = create_user_pref_vector(user_pref)
    
    # Calculate Euclidean distances
    distances = euclidean_distances(data, user_vector.reshape(1, -1)).flatten()
    
    # Find the k smallest distances (most similar)
    top_k_unsorted = np.argpartition(distances, k)[:k]
    sorted_indices = top_k_unsorted[np.argsort(distances[top_k_unsorted])]
    
    # Retrieve and return the top-k most similar businesses
    recommendations = original_df.iloc[sorted_indices][[
        "name", "stars", "attributes_RestaurantsPriceRange2", "address", "city",
        "attributes_GoodForKids", "attributes_Ambience_romantic", "attributes_Ambience_intimate",
        "attributes_Ambience_touristy", "attributes_Ambience_trendy", "attributes_Ambience_classy",
        "attributes_Ambience_casual", "Bars_Night", "Beauty_Health_Care", "Cafes",
        "GYM", "Restaurants_Cuisines", "Shops"
    ]].copy()
    
    recommendations["euclidean_distance"] = distances[sorted_indices]
    recommendations["similarity_score"] = 1 / (1 + recommendations["euclidean_distance"])
    
    return recommendations

In [22]:
recommend_businesses_euclidean(user_pref, ds, data_set)

,name,stars,attributes_RestaurantsPriceRange2,address,city,attributes_GoodForKids,attributes_Ambience_romantic,attributes_Ambience_intimate,attributes_Ambience_touristy,attributes_Ambience_trendy,attributes_Ambience_classy,attributes_Ambience_casual,Bars_Night,Beauty_Health_Care,Cafes,GYM,Restaurants_Cuisines,Shops,euclidean_distance,similarity_score
13488,Topgolf,3.5,3,500 Cowan St,Nashville,1,0,0,0,1,0,1,1,0,0,0,1,0,1.000000,0.500000
5249,Frothy Monkey - Nashville,4.0,2,1400 51st Ave N,Nashville,1,0,0,0,1,0,1,1,0,1,0,1,0,1.118034,0.472136
10383,Frothy Monkey,4.0,2,125 5th Ave S,Franklin,1,0,0,0,1,0,1,1,0,1,0,1,0,1.118034,0.472136


# FAISS

In [32]:
def recommend_businesses_faiss_cosine(user_pref, data, original_df, k=3):
    import numpy as np

    data_np = np.ascontiguousarray(data.to_numpy().astype('float32'))
    user_vector = np.ascontiguousarray(create_user_pref_vector(user_pref).astype('float32').reshape(1, -1))
    faiss.normalize_L2(data_np)
    faiss.normalize_L2(user_vector)
    index = faiss.IndexFlatIP(data_np.shape[1])
    index.add(data_np)

    distances, indices = index.search(user_vector, k)

    sorted_indices = indices[0]

    recommendations = original_df.iloc[sorted_indices][[
        "name", "stars", "attributes_RestaurantsPriceRange2", "address", "city",
        "attributes_GoodForKids", "attributes_Ambience_romantic", "attributes_Ambience_intimate",
        "attributes_Ambience_touristy", "attributes_Ambience_trendy", "attributes_Ambience_classy",
        "attributes_Ambience_casual", "Bars_Night", "Beauty_Health_Care", "Cafes",
        "GYM", "Restaurants_Cuisines", "Shops"
    ]].copy()

    recommendations["cosine_similarity"] = distances[0]

    return recommendations


In [33]:
recommend_businesses_faiss_cosine(user_pref, ds, data_set, k=5)

,name,stars,attributes_RestaurantsPriceRange2,address,city,attributes_GoodForKids,attributes_Ambience_romantic,attributes_Ambience_intimate,attributes_Ambience_touristy,attributes_Ambience_trendy,attributes_Ambience_classy,attributes_Ambience_casual,Bars_Night,Beauty_Health_Care,Cafes,GYM,Restaurants_Cuisines,Shops,cosine_similarity
13488,Topgolf,3.5,3,500 Cowan St,Nashville,1,0,0,0,1,0,1,1,0,0,0,1,0,0.981480
9960,Stones Throw at Armature Works,4.0,3,1910 North Ola Avenue,Tampa,1,0,0,0,1,0,1,1,0,0,0,1,0,0.979296
5451,Mr and Mrs Crab,4.0,3,2042 Badlands Dr,Brandon,1,0,0,0,1,0,1,1,0,0,0,1,0,0.979296
9797,Hanjan,4.0,2,3735 99 Street NW,Edmonton,1,0,0,0,1,0,1,1,0,1,0,1,0,0.976795
7703,Wabi Sabi Japanese Fusion Cafe,4.0,2,35583 US Hwy 19 N,Palm Harbor,1,0,0,0,1,0,1,1,0,1,0,1,0,0.976795
